In [13]:
workspace()
include("PixelArts/PixelArts.jl")
include("RL/RL.jl")
using RL
using PixelArts

In [14]:
right_turn, colour_array = get_env("right_turn")

(RL.RLEnv(Array{Int64,1}[[1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [1, 10]  …  [18, 8], [18, 9], [18, 10], [18, 11], [18, 12], [18, 13], [18, 14], [18, 15], [18, 16], [18, 17]], RL.actions, RL.step, RL.task_end), String["green" "green" … "green" "green"; "green" "green" … "green" "green"; … ; "green" "green" … "green" "green"; "green" "green" … "green" "green"])

In [15]:
println(fieldnames(RLEnv))
println(fieldnames(RLAgent))

Symbol[:states, :actions, :step, :task_end]
Symbol[:policy, :state]


In [16]:
println("States ", right_turn.states)
println("Actions ", right_turn.actions([1,1]))
println("step ", right_turn.step([1,1], "right"))
println("task_end ", right_turn.task_end([1,1]))

States Array{Int64,1}[[1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [1, 10], [1, 11], [1, 12], [1, 13], [1, 14], [1, 15], [1, 16], [1, 17], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [2, 10], [2, 11], [2, 12], [2, 13], [2, 14], [2, 15], [2, 16], [2, 17], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [3, 9], [3, 10], [3, 11], [3, 12], [3, 13], [3, 14], [3, 15], [3, 16], [3, 17], [4, 1], [4, 2], [4, 3], [4, 4], [4, 5], [4, 6], [4, 7], [4, 8], [4, 9], [4, 10], [4, 11], [4, 12], [4, 13], [4, 14], [4, 15], [4, 16], [4, 17], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6], [5, 7], [5, 8], [5, 9], [5, 10], [5, 11], [5, 12], [5, 13], [5, 14], [5, 15], [5, 16], [5, 17], [6, 1], [6, 2], [6, 3], [6, 4], [6, 5], [6, 6], [6, 7], [6, 8], [6, 9], [6, 10], [6, 11], [6, 12], [6, 13], [6, 14], [6, 15], [6, 16], [6, 17], [7, 1], [7, 2], [7, 3], [7, 4], [7, 5], [7, 6], [7, 7], [7, 8], [7, 9], [7, 10], [7, 11], [7, 12], [7, 13], [7, 14

In [17]:
rtcanvas = create_canvas(size(colour_array)..., 400, 400)
add_pixels(rtcanvas, colour_array)

In [18]:
policy(state) = rand(right_turn.actions(state))
init_state = [18, 3]
agent = RLAgent(policy, init_state)

RL.RLAgent(policy, [18, 3])

In [19]:
s, r, a = episode!(agent, right_turn, verbose = true)

step: 0, choosing left, moving to state [18, 2], obtaining reward -10
step: 1, choosing down, moving to state [18, 2], obtaining reward -10
step: 2, choosing up, moving to state [17, 2], obtaining reward -10
step: 3, choosing up, moving to state [16, 2], obtaining reward -10
step: 4, choosing right, moving to state [16, 3], obtaining reward -1
step: 5, choosing down, moving to state [17, 3], obtaining reward -1
step: 6, choosing left, moving to state [17, 2], obtaining reward -10
step: 7, choosing left, moving to state [17, 1], obtaining reward -10
step: 8, choosing down, moving to state [18, 1], obtaining reward -10
step: 9, choosing up, moving to state [17, 1], obtaining reward -10
step: 10, choosing right, moving to state [17, 2], obtaining reward -10
step: 11, choosing up, moving to state [16, 2], obtaining reward -10
step: 12, choosing down, moving to state [17, 2], obtaining reward -10
step: 13, choosing left, moving to state [17, 1], obtaining reward -10
step: 14, choosing right

(Array{Int64,1}[[18, 3], [18, 2], [18, 2], [17, 2], [16, 2], [16, 3], [17, 3], [17, 2], [17, 1], [18, 1]  …  [2, 10], [2, 11], [2, 12], [2, 11], [3, 11], [3, 12], [2, 12], [2, 13], [2, 12], [1, 12]], Any[-10, -10, -10, -10, -1, -1, -10, -10, -10, -10  …  -10, -1, -1, -1, -1, -1, -1, -1, -1, 0], Any["left", "down", "up", "up", "right", "down", "left", "left", "down", "up"  …  "left", "right", "right", "left", "down", "right", "up", "right", "left", "up"])

In [20]:
racer = add_pixel_cross(rtcanvas, 17, 4, "blue")
for x in s
    translate_element(racer, x...)
    sleep(0.1)
end
    

In [26]:
Q = zeros(size(right_turn.states, 1), 4)
greedy = Dict()
n = 3
for (i, x) in enumerate(right_turn.states)
    action_set = right_turn.actions(x)
    for (j, action) in enumerate(action_set)
        totalr = 0
        for k in 1:n
            s, r, a = episode(agent, right_turn, x, action)
            totalr += (sum(r) - totalr) / k
        end
        Q[i, j] = totalr
    end
    val, k = findmax(Q[i,:])
    greedy[x] = action_set[k]
end

In [27]:
Q

306×4 Array{Float64,2}:
  -5053.33    -1499.67   -6530.0     -4375.67  
 -10535.0    -11222.3   -11411.0    -13768.3   
 -13432.7     -3152.67  -10142.0     -6950.33  
 -11263.7     -7339.0   -12569.7     -7657.33  
  -1320.0     -4208.33  -17038.3     -6484.0   
  -4992.33    -5713.67   -8272.67    -1908.0   
  -2335.33    -4682.33   -4311.67    -9156.0   
 -14083.7     -4171.67   -5529.0     -5414.33  
  -4626.33    -1080.0    -7401.0       -23.3333
  -3949.33    -1202.67   -8401.33    -2421.67  
   -305.667   -1749.0     -861.333       0.0   
      0.0     -2739.67   -2750.0         0.0   
      0.0     -7479.33       0.0         0.0   
      ⋮                                        
  -4648.33    -5514.0    -9215.33   -10870.3   
 -16387.7     -2711.33   -3154.0     -5363.33  
  -5629.0     -9983.33   -8317.33    -7536.33  
  -4350.0     -4205.67   -4494.0     -8714.33  
  -6406.33   -12465.7    -3291.33    -5083.0   
  -6651.67    -7206.0    -6031.67    -6217.33  
 -16970.0     -4

In [29]:
greedy

Dict{Any,Any} with 306 entries:
  [5, 10]  => "left"
  [6, 9]   => "down"
  [17, 13] => "left"
  [3, 2]   => "left"
  [9, 8]   => "down"
  [1, 7]   => "up"
  [11, 12] => "left"
  [5, 17]  => "up"
  [8, 3]   => "up"
  [1, 15]  => "up"
  [12, 5]  => "down"
  [14, 4]  => "right"
  [1, 17]  => "up"
  [3, 4]   => "up"
  [16, 14] => "down"
  [9, 17]  => "left"
  [4, 14]  => "left"
  [17, 4]  => "down"
  [1, 3]   => "down"
  [12, 17] => "up"
  [13, 11] => "down"
  [2, 1]   => "right"
  [1, 4]   => "down"
  [7, 17]  => "up"
  [4, 2]   => "down"
  ⋮        => ⋮

In [ ]:
rotation = Dict("up" => 90, "down" => 270, "right" => 0, "left" => 180)
for s in right_turn.states add_pixel_arrow(rtcanvas, s..., rotation[greedy[s]]) end

In [21]:
n = 10
greedy = Dict()
for outi in 1:25
    Q = zeros(size(right_turn.states, 1), 4)
    greedy = Dict()
    for (i, x) in enumerate(right_turn.states)
        action_set = right_turn.actions(x)
        for (j, action) in enumerate(action_set)
            totalr = 0
            for k in 1:n
                s, r, a = episode(agent, right_turn, x, action)
                totalr += 0.25 * (sum(r) - totalr) 
            end
            Q[i, j] = totalr
        end
        val, k = findmax(Q[i,:])
        greedy[x] = action_set[k]
    end
    old_greedy = copy(greedy)
    agent.policy(s) = (rand() < 0.1) ? rand(right_turn.actions(s)) : old_greedy[s]
end

In [22]:
rtcanvas2 = create_canvas(size(colour_array)..., 400, 400)
add_pixels(rtcanvas2, colour_array)

In [23]:
rotation = Dict("up" => 90, "down" => 270, "right" => 0, "left" => 180)
for s in right_turn.states add_pixel_arrow(rtcanvas2, s..., rotation[greedy[s]]) end

In [24]:
racer = add_pixel_cross(rtcanvas2, 17, 4, "blue")
s, r, a = episode(agent, right_turn, [13, 3])
for x in s
    translate_element(racer, x...)
    sleep(0.1)
end

In [12]:
Q

LoadError: [91mUndefVarError: Q not defined[39m

In [ ]:
rtcanvas2 = create_canvas(size(colour_array)..., 400, 400)
add_pixels(rtcanvas2, colour_array)